In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from math import ceil
from tqdm import tqdm_notebook
import tqdm
from collections import Counter
from operator import itemgetter
from datetime import date, datetime

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import json

## Autonews

In [ ]:
link = 'https://www.autonews.ru/ajax/tags/?tag=%D0%9D%D0%BE%D0%B2%D0%BE%D1%81%D1%82%D0%B8&offsetrbc=0&offset=500&limit=500'
r = requests.get(link)
soup = BS(r.json()['html'])

dates = [(div.find('span', 'item-medium__date').text, div.find('a', 'item-medium__link')['href']) for div in soup.find_all('div', 'item-medium')]
dates

In [ ]:
autonews_20 = defaultdict(list)
for i in dates[1:]:
    if 'января' in i[0]:
        date = '2020.01.'+i[0].split()[0]
    if 'февраля' in i[0]:
        date = '2020.02.'+i[0].split()[0]
    autonews_20[date].append(i[1])
autonews_20

In [ ]:
autonews_2020 = []
for date in tqdm_notebook(sorted(list(autonews_20.keys()))):
    for link in tqdm_notebook(autonews_20[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            autonews_2020.append({'Media':'autonews.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.find('div', 'article__header__title').text, ' '.join([p.text for p in soup.find('div', 'article__text').find_all('p') if p.text]).replace('\xa0', ' ')]), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(autonews_2020).to_csv('autonews_2020.csv', encoding='utf-8')
pd.DataFrame(autonews_2020).to_csv('autonews_2020.csv', encoding='utf-8')
pd.DataFrame(autonews_2020)

## Za Rulyem

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
zarulem_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        zarulem_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.zr.ru%20OR%20domainis:zr.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                zarulem_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
                
with open('zarulem_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(zarulem_20, f)

In [ ]:
zarulem_2020 = []
for date in tqdm_notebook(list(zarulem_20.keys())):
    for link in tqdm_notebook(zarulem_20[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            zarulem_2020.append({'Media':'zr.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text.strip() for p in soup.find('div', 'stroy_content').find_all('p') if p.text.strip() and 'Фото' not in p.text])]).replace('\xa0', ' '),
                                 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(zarulem_2020).to_csv('zarulem_2020.csv', encoding='utf-8')
pd.DataFrame(zarulem_2020).to_csv('zarulem_2020.csv', encoding='utf-8')
pd.DataFrame(zarulem_2020)

## Autoreview

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
review_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        review_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.autoreview.ru%20OR%20domainis:autoreview.ru)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                review_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
                
with open('review_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(review_20, f)

In [ ]:
review_2020 = []
for date in tqdm_notebook(list(review_20.keys())):
    for link in tqdm_notebook(review_20[date]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            review_2020.append({'Media':'autoreview.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'full-article').find_all('p')])]).replace('\xa0', ' '),
                                 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(review_2020).to_csv('review_2020.csv', encoding='utf-8')
pd.DataFrame(review_2020).to_csv('review_2020.csv', encoding='utf-8')
pd.DataFrame(review_2020)

## Motor

In [ ]:
def get_article_motor(link):
    r = requests.get(link)
    soup = BS(r.text)
    
    try:
        title = soup.h1.div.text
        lead = soup.find('div', 'lead').div.text
        text = ' '.join([p.text for div in soup.find_all('div', 'text') for p in div.find_all('p')])
        return ' .'.join([title, lead, text])
    except (AttributeError, IndexError):
        title = soup.h1.div.text
        text = ' '.join([p.text for div in soup.find_all('div', 'text') for p in div.find_all('p')])
        return ' .'.join([title, text])

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
motor_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020/{month:02d}/{day+1:02d}'
        try:
            url = f'https://motor.ru/pulse/{date}'
            r = requests.get(url)
            soup = BeautifulSoup(r.text)
            links = ['https://motor.ru'+a['href'] for a in soup.find_all('div', 'block')[1].find_all('a') 
                     if 'authors' not in a['href'] and 'pulse' not in a['href']]
            motor_20[f'2020.{month:02d}.{day+1:02d}'] = links
            print(f'2020.{month:02d}.{day+1:02d}', len(links))
        except (AttributeError, IndexError):
            print(date)
with open('motor_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(motor_20, f)

In [ ]:
motor_2020 = []
for date in tqdm_notebook(list(motor_20.keys())):
    for link in tqdm_notebook(motor_20[date]):
        try:
            motor_2020.append({'Media':'motor.ru', 'Date':date, 'Link':link,
                                       'Text':get_article_motor(link), 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(motor_2020).to_csv('motor_2020.csv', encoding='utf-8')
pd.DataFrame(motor_2020).to_csv('motor_2020.csv', encoding='utf-8')
pd.DataFrame(motor_2020)

## Sport Express

In [ ]:
dates = defaultdict(list)
for page in tqdm_notebook(range(282, 489)):
    link = f'https://www.sport-express.ru/search/page{page}/?sw=%F7%F2%EE&sportType=%C2%E8%E4+%F1%EF%EE%F0%F2%E0&kolonki=%CA%EE%EB%EE%ED%EA%E8&dt_to=&tp%5B0%5D=news'
    r = requests.get(link)
    soup = BS(r.text)
    for item in [(div.find('a', 'se19-search-item__title')['href'], 
                  div.find('p', 'se19-search-item__date').text.split(',')[0])
                 for div in soup.find('div', 'se19-search__results').find_all('div', 'se19-search-item')]:
        if 'января' in item[1]:
            dates[f'{item[1].split()[2]}.01.{item[1].split()[0]}'].append(item[0])
        if 'февраля' in item[1]:
            dates[f'{item[1].split()[2]}.02.{item[1].split()[0]}'].append(item[0])

In [ ]:
express_2020 = []
for date in tqdm_notebook(sorted(list(dates.keys()))):
    if len(date) == 9:
        date = date[:-1]+'0'+date[-1]
    for link in tqdm_notebook(dates[date]):
#         if link not in pd.DataFrame(echo_2020).Link:
        try:
            r = requests.get(link)
            soup = BS(r.text)
            express_2020.append({'Media':'sport-express.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'article_text').find_all('p')])]).replace('\xa0', ' '),
                                 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(express_2020).to_csv('express_2020.csv', encoding='utf-8')
pd.DataFrame(express_2020).to_csv('express_2020.csv', encoding='utf-8')
pd.DataFrame(express_2020)

## Championat

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
times = [['000000', '120000'], ['120000', '235959']]
championat_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020{month:02d}{day+1:02d}'
        championat_20[f'2020.{month:02d}.{day+1:02d}'] = []
        for time in times:
            try:
                url = f'https://api.gdeltproject.org/api/v2/doc/doc?query=%20(domain:.championat.com%20OR%20domainis:championat.com)%20sourcelang:rus&mode=ArtList&maxrecords=250&sort=DateDesc&format=html&startdatetime={date}{time[0]}&enddatetime={date}{time[1]}'
                r = requests.get(url)
                soup = BeautifulSoup(r.text)
                championat_20[f'2020.{month:02d}.{day+1:02d}'].extend([a['href'] for a in soup.body.table.find_all('a')])
                print(f'2020.{month:02d}.{day+1:02d}', len(soup.body.table.find_all('a')))
            except (AttributeError, IndexError):
                print(date)
with open('championat_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(championat_20, f)

In [ ]:
championat_2020 = []
for date in tqdm_notebook(list(championat_20.keys())):
    for link in tqdm_notebook(championat_20[date]):
        try:
            r = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0'})
            soup = BeautifulSoup(r.text)
            championat_2020.append({'Media':'championat.ru', 'Date':date, 'Link':link,
                                  'Text':' '.join([soup.find('div', 'article-head__title').text, 
                                                   ' '.join([p.text for p in 
                                                             soup.find('div', 'article-content').find_all('p')])]), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(championat_2020).to_csv('championat_2020.csv', encoding='utf-8')
pd.DataFrame(championat_2020).to_csv('championat_2020.csv', encoding='utf-8')
pd.DataFrame(championat_2020)

## Sports.Ru

In [ ]:
def get_article_sports(link):
    r = requests.get(link)
    soup = BS(r.text)

    title = soup.h1.text.strip()
    text = ' '.join([p.text for p in soup.find('div', 'news-item__content').find_all('p') if 'Facebook' not in p.text][:-1])
    return ' .'.join([title, text])

In [ ]:
months = {1:31, 2:28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9:30, 10:31, 11:30, 12:31}
sports_20 = dict()
for month in tqdm_notebook(list(months.keys())[:2]):
    for day in tqdm_notebook(list(range(months[month]))):
        date = f'2020/{month}/{day+1:02d}'
#         sports_20[f'2020.{month:02d}.{day+1:02d}'] = []
        try:
            url = f'https://www.sports.ru/stat/news/{date}.html'
            r = requests.get(url)
            soup = BeautifulSoup(r.text)
            links = ['https://www.sports.ru'+li.a['href'] for li in 
                     soup.find('ul', 'archive-list').find_all('li') if 'https:' not in li.a['href']]
            sports_20[f'2020.{month:02d}.{day+1:02d}'] = links
            print(f'2020.{month:02d}.{day+1:02d}', len(links))
        except (AttributeError, IndexError):
            print(date)
with open('sports_links_2020.json', 'w', encoding = 'utf-8') as f:
    json.dump(sports_20, f)

In [ ]:
sports_2020 = []
for date in tqdm_notebook(list(sports_20.keys())):
    for link in tqdm_notebook(sports_20[date]):
        try:
            sports_2020.append({'Media':'sports.ru', 'Date':date, 'Link':link,
                                       'Text':get_article_sports(link), 'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(sports_2020).to_csv('sports_2020.csv', encoding='utf-8')
pd.DataFrame(sports_2020).to_csv('sports_2020.csv', encoding='utf-8')
pd.DataFrame(sports_2020)

## N+1

In [ ]:
texts = ['%D0%BE%D0%B1%D0%BD%D0%B0%D1%80%D1%83%D0%B6%D0%B8%D0%BB%D0%B8', # обнаружили
        '%D0%B8%D1%81%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5', # исследование
        '%D1%81%D1%82%D0%B0%D1%82%D1%8C%D1%8F', # статья
         '%D0%B6%D1%83%D1%80%D0%BD%D0%B0%D0%BB%D0%B5', # журнале
         '%D1%83%D1%87%D0%B5%D0%BD%D1%8B%D0%B5'] # ученые
         '%D0%BD%D0%B0%D1%88%D0%BB%D0%B8', # нашли
        ]
links = set()
for text in texts:
    link = f'https://nplus1.ru/search?q={text}'
    r = requests.get(link)
    soup = BS(r.text)
    links.update(['https://nplus1.ru'+a['href'] for a in soup.find('ol', 'list-search-results').find_all('a')
 if '2019' in a['href'] or '2020/01' in a['href'] or '2020/02' in a['href']])

In [ ]:
nplus1_19_20 = []
for link in tqdm_notebook(links):
    if '2020' in link:
        date = link[link.index('/2020/')+1:link.index('/2020/')+11].replace('/', '.')
    if '2019' in link:
        date = link[link.index('/2019/')+1:link.index('/2019/')+11].replace('/', '.')    
    try:
        r = requests.get(link)
        soup = BS(r.text)
        nplus1_19_20.append({'Media':'nplus1.ru', 'Date':date, 'Link':link,
                              'Text':'. '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'body js-mediator-article').find_all('p') if p.text and 'class' not in p.attrs]).replace('\xa0', ' ')]), 
                              'Dep':'', 'Tags':[]})
    except (requests.exceptions.ConnectionError, AttributeError, 
            IndexError, requests.exceptions.TooManyRedirects, ValueError):
        print(link)
if date.endswith('1'):
    pd.DataFrame(nplus1_19_20).to_csv('nplus1_19_20.csv', encoding='utf-8')
pd.DataFrame(nplus1_19_20).to_csv('nplus1_19_20.csv', encoding='utf-8')
pd.DataFrame(nplus1_19_20)

In [ ]:
nplus1_19_20 = pd.DataFrame(nplus1_19_20).sort_values(by='Date').reset_index().drop(['index'], axis = 1)
nplus1_19_20.to_csv('nplus1_19_20_order.csv', encoding='utf-8')

## Postnauka

In [ ]:
months_names = {'Января':'01', 'Февраля':'02', 'Марта':'03', 'Апреля':'04', 
                'Мая':'05', 'Июня':'06', 'Июля':'07', 'Августа':'08', 
                'Сентября':'09', 'Октября':'10', 'Ноября':'11', 'Декабря':'12'}
postnauka_19_20 = []
for page in tqdm_notebook(range(151)):
    url = f'https://postnauka.ru/api/v4/grid?page={page+1}'
    r = requests.get(url)
    for link in tqdm_notebook(['https://postnauka.ru'+i['url'] for i in r.json() if 'courses' not in i['url']]):
        try:
            r = requests.get(link)
            soup = BS(r.text)
            date = soup.find('div', 'pub-data-item').text
            if '2019' in date or ('2020' in date and 
                              (months_names[date.split()[1]]=='01' or months_names[date.split()[1]]=='02')):
                postnauka_19_20.append({'Media':'postnauka.ru', 'Date':f'{date.split()[2]}.{months_names[date.split()[1]]}.{date.split()[0]}', 'Link':link,
                                  'Text':' '.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'article-text').find_all('p')])]).replace('\xa0', ' ').replace('\n', ' '), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(postnauka_19_20).to_csv('postnauka_19_20.csv', encoding='utf-8')
pd.DataFrame(postnauka_19_20).to_csv('postnauka_19_20.csv', encoding='utf-8')
pd.DataFrame(postnauka_19_20)

## Indicator.Ru

In [ ]:
links = defaultdict(list)
rubrics = ['astronomy', 'biology', 'humanitarian-science', 'mathematics', 'earth-science',
          'agriculture', 'medicine', 'engineering-science', 'physics', 'chemistry-and-materials']
for rubric in tqdm_notebook(rubrics):
    link = f'https://indicator.ru/api/v2/topics?include=image&limit=100&offset=0&rubric_root={rubric}&topic_type=-daily_photo%2Cevent&with_filtered_count=1'
    r = requests.get(link)
    print(rubric, r.json()['meta']['filtered_count'], '\n')
    num_of_pages = ceil(r.json()['meta']['filtered_count']/100)
    for i in tqdm_notebook(range(num_of_pages)):
        link = f'https://indicator.ru/api/v2/topics?include=image&limit=100&offset={i*100}&rubric_root={rubric}&topic_type=-daily_photo%2Cevent&with_filtered_count=1'
        r = requests.get(link)
        for item in r.json()['data']:
            date = item['attributes']['published_at'][:10]
            if '2019' in date or '2020-01' in date or '2020-02' in date:
#                 print(date.replace('-', '.'), )
                links[date.replace('-', '.')].append('https://indicator.ru'+item['attributes']['link'])
    print(len(links))

In [ ]:
links_indicator_19_20 = defaultdict(list)
links_without_date = []
for link in links:
    if link.endswith('2019.htm') or link.endswith('01-2020.htm') or link.endswith('02-2020.htm'):
        date = link[-14:-4].split('-')
        date = f'{date[2]}.{date[1]}.{date[0]}'
        links_indicator_19_20[date].append(link)
    if not (link.endswith('2020.htm') or link.endswith('2019.htm') or link.endswith('2018.htm')
           or link.endswith('2017.htm') or link.endswith('2016.htm')):
        links_without_date.append(link)

In [ ]:
indicator_19_20 = []
for date in tqdm_notebook(sorted(list(links.keys()))):
    for link in tqdm_notebook(links[date]):
        time.sleep(1)
        try:
            r = requests.get(link)
            soup = BS(r.text)
            indicator_19_20.append({'Media':'indicator.ru', 'Date':date, 'Link':link,
                                  'Text':' .'.join([soup.h1.text, ' '.join([p.text for p in soup.find('div', 'text').find_all('p') 
                                   if 'Мои источники' not in p.text and 'Подписывайтесь на Indicator.Ru в соцсетях' not in p.text])]), 
                                  'Dep':'', 'Tags':[]})
        except (requests.exceptions.ConnectionError, AttributeError, 
                IndexError, requests.exceptions.TooManyRedirects, ValueError):
            print(link)
    if date.endswith('1'):
        pd.DataFrame(indicator_19_20).to_csv('indicator_19_20.csv', encoding='utf-8')
pd.DataFrame(indicator_19_20).to_csv('indicator_19_20.csv', encoding='utf-8')
pd.DataFrame(indicator_19_20)